In [3]:
# !pip3 install selenium

     ---------------------------------------- 6.5/6.5 MB 11.9 MB/s eta 0:00:00
     -------------------------------------- 140.9/140.9 kB 8.7 MB/s eta 0:00:00
     -------------------------------------- 155.3/155.3 kB 9.1 MB/s eta 0:00:00
     ------------------------------------- 384.9/384.9 kB 12.1 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB ? eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
#!pip3 install webdriver_manager

     ---------------------------------------- 62.5/62.5 kB ? eta 0:00:00
     ---------------------------------------- 77.1/77.1 kB ? eta 0:00:00
     ---------------------------------------- 97.1/97.1 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import time

import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [3]:
# 수집할 맥주 목록
beer_list = ['Apostel Brau',
 'Egger Zwickl', 'Egger Marzenbier', 'Holsten Premium Beer', 'Franzisaner Hefe-Weissbier',
 'Egger Radler Grapefruit', 'Barvaria Premium', 'Barvaria 8.6', 'Lapin Kulta IV A',
 'Grolsch Premium Larger', 'Gambrinus Original', 'XXXX Gold', 'Leffe Brown',
 'Lowenbrau Original', 'Asahi Super dray Black ', 'Harbin Beer', "beck's",
 'Hoegaarden Rosee', 'Platinum White Ale', 'Platinum Pale Ale', 'Ambar Especial Larger',
 'Schöfferhofer Grapefruit', 'Volfas Engelman Grünberger Hefeweizen', 'Berliner Kindl Pilsener ',
 'BURGE MEESTER', 'Red Rock', 'Erdinger Dunkel', 'Warsteiner Premium Verum', "Queen's Ale Blonde Type"]

# 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

In [4]:
beer_list

,검색이름
0,Apostel Brau
1,Egger Zwickl
2,Egger Marzenbier
3,Holsten Premium Beer
4,Franzisaner Hefe-Weissbier
5,Egger Radler Grapefruit
6,Barvaria Premium
7,Barvaria 8.6
8,Lapin Kulta IV A
9,Grolsch Premium Larger


In [5]:
# 데이터 프레임 생성
data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

# chromedriver.exe 파일 경로 설정
#chromedriver = 'chromedriver.exe'
# 크롤링 할 경로 설정
url = 'https://www.ratebeer.com/search?tab=beer'

# 셀레니움으로 웹브라우저를 오픈합니다.
browser = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
browser.get(url)
browser.set_window_size(900, 900)
time.sleep(1)

In [8]:
def crawl(browser, beer, data, k):
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    browser = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
    url='https://www.ratebeer.com/'
    browser.get(url)
    browser.set_window_size(900, 900)
    # 1번 사진에 해당 : 맥주 검색
    time.sleep(2)
    element = browser.find_element(By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div[2]/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)
    # 2번 사진에 해당 : 상품 선택
    browser.find_element(By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div[2]/div[2]/a[1]/div/div[2]').click()

    # 3번 사진에 해당 : 상품 이름 수집
    time.sleep(3)
    beer_name = browser.find_element(By.CSS_SELECTOR,'.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text

    error_cnt = 0

    while 1:
        try :
            # 4번 사진에 해당 : 전체 리뷰 개수 수집
            time.sleep(3)
            string = browser.find_element(By.CLASS_NAME,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text

            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # 5번 사진에 해당 : Score breakdown 클릭
    time.sleep(3)
    element = browser.find_element(By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    time.sleep(3)
    # 해당 element로 이동하는 코드입니다. 반드시 적어주세요.
    browser.execute_script("arguments[0].click();", element)

    # 수집할 Page 수를 계산합니다.
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i, '번째 페이지입니다.')

        # 6번 사진에 해당 : 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        beer_info = browser.find_element(By.CSS_SELECTOR,'.px-4.fj-s.f-wrap').text
        tmp = []

        # 수집한 것을 데이터프레임에 저장
        tmp.append(beer_info)
        tmp = pd.DataFrame(data=tmp, columns=['맥주정보'])
        tmp['맥주이름'] = beer_name
        tmp['검색이름'] = beer
        data = pd.concat([data, tmp])

        # 다음 페이지로 넘어가기 : 7번 사진에 해당합니다.
        # div, span, title 태그 후 속성은 class 외에도 사용 가능
        try :
            element = browser.find_element(By.XPATH,'//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(3)
            browser.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    # 데이터가 중복 수집될 경우 리뷰 수 만큼만 Cut
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))

    # 데이터를 csv, excel 파일로 저장합니다.
    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("beer_n_"+str(k+47)+".csv", encoding='utf-8')

    browser.quit()

    return result


In [9]:
# 맥주 리스트 개수만큼 데이터 수집
for k in range(len(beer_list)):
    result = crawl(browser, beer_list['검색이름'].iloc[k], data, k)

url_open... Apostel Brau 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
0 번째 페이지입니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
리뷰수 :  156 수집된 리뷰수 :  11
url_open... Egger Zwickl 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
0 번째 페이지입니다.
리뷰수 :  10 수집된 리뷰수 :  1
url_open... Egger Marzenbier 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
0 번째 페이지입니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페이지입니다.
13 번째 페이지입니다.
리뷰수 :  207 수집된 리뷰수 :  14
url_open... Holsten Premium Beer 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
0 번째 페이지입니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페이지입니다.
13 번째 페이지입니다.
14 번째 페이지입니다.
15 번째 페이지입니다.
16 번째 페이지입니다.
17 번째 페이지입니다.
18 번째 페이지입니다.
19 번째 페이지입니다.
20 번째 페이지입니다.
21 번째 페이지입니다.
22 번째 페이지입니다.
23 번째 페이지입니다

In [ ]:
df1 = pd.read_csv('C:\Users\10PRO\beer_n_0.csv')